In [ ]:
from pandas import *
from numpy import *

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential,Model
from keras.layers import LSTM,Dense,GRU,Bidirectional,Input,Multiply,Reshape,Average,concatenate,Activation
from sklearn.metrics import mean_squared_error as mse
from keras import backend as k
import random as rn

from sklearn.model_selection import KFold# 处理交叉验证中训练集和测试集的划分(一般横截面数据)
from sklearn.model_selection import TimeSeriesSplit # 处理时序数据的交叉验证问题时的训练集和测试集

rn.seed(12345)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.random.set_seed(1234)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)


def kn(x0,X):
    k=[]
    for i in range(0,X.shape[0],1):
        y=sum(square(x0-X[i,:]))
        k.append(y)
    k=array(k)
    min=k.min()
    max=k.max()
    for j in range(0,k.shape[0],1):
        k[j]=40*(k[j]-min)/(max-min)
        
    return exp((-0.5)*k)


shuju=read_csv('input/ceshi/ETTh1.csv')# 导入数据，此时数据格式为DataFrame
shuju[shuju==0.0]=NaN  # 将0值转化为缺失值NaN
shuju.isnull().sum()# 查看缺失情况
shuju.fillna(shuju.median(),inplace=True) # 使用中位数填补缺失值
shuju.isnull().sum()# 检查填补情况
dataset=array(shuju)[:,1:].astype('float')
real_mse=[]
real_kmse=[]

# 使用一定量的训练集预测6的点，固定测试集长度为6，固定窗宽，滑动100次

samples=[1000,2500,5000,10000,17420]# 观察样本量变化对试验结果的影响
sizetrain=[400,1900,4400,9400,16820]# 计算满足 (size+6)+99*6=sample
sizebatch=[256,1024,2048,4096,4096*2]# batchsize的大小
lb=list(range(0,len(samples),1)) # 指标集合
for index in lb:
    label=samples[index] # 样本数
    n=sizetrain[index]# 训练集长度
    m=sizebatch[index]# batchsize
    
    newdata=dataset[0:label,:]
    mse=[]
    kmse=[]

    for i in range(1,101,1):
        train_data=newdata[(i-1)*6:n+(i-1)*6,:]
        test_data=newdata[n+(i-1)*6:n+i*6,:]
        
        # 使用一般的lstm处理
        train_x=train_data[:,:-1]
        rtrain_x=reshape(train_x,(train_x.shape[0],1,train_x.shape[1]))
        train_y=train_data[:,-1:]
        # train_y 不用变
        
        test_x=test_data[:,:-1]
        rtest_x=reshape(test_x,(test_x.shape[0],1,test_x.shape[1]))
        test_y=test_data[:,-1:]
        # test_y 不用变
        
        batch_size=None
        timesteps=1
        input_dim=train_x.shape[1]
        
        input_layer=Input(batch_shape=(batch_size,timesteps,input_dim))
        d1=Bidirectional(LSTM(16,return_sequences=True))(input_layer)
        d2=Bidirectional(LSTM(8,return_sequences=True))(d1)
        d=Bidirectional(LSTM(8,return_sequences=False))(d2)
        output_layer=Dense(1)(d)
        model=Model(inputs=input_layer,outputs=output_layer)
        optimizer = tf.keras.optimizers.RMSprop(0.001)
        model.compile(loss='mse',optimizer=optimizer)
        model.fit(rtrain_x,train_y,shuffle=False,verbose=1,validation_data=None,batch_size=m,epochs=50)
        yhat=model.predict(rtest_x)
        mse.append(mean(square(yhat[:,0]-test_y[:,0])))
        
        # 使用kernel方法+lstm处理
        kyhat=[]
        
        # test_data.shape[0]恒等于1，所以此处j恒等于0
        for j in range(0,test_data.shape[0],1):
            ktest_x=test_data[j:j+1,:-1]
            krtest_x=reshape(ktest_x,(ktest_x.shape[0],1,ktest_x.shape[1]))
            ktest_y=test_data[j:j+1,-1:]
            # ktest_y 不用变
            
            def newmse(y_true,y_pred):
                y=k.mean(kn(ktest_x,train_x)*k.square(y_true-y_pred),axis=-1)
                return y
            
            model1=Model(inputs=input_layer,outputs=output_layer)
            model1.compile(loss=newmse,optimizer=optimizer)
            model1.fit(rtrain_x,train_y,shuffle=False,verbose=1,validation_data=None,batch_size=m,epochs=50)
            kyhat.append(model1.predict(krtest_x)[0,0])
        kernel_yhat=array(kyhat)
        kmse.append(mean(square(kernel_yhat-test_y[:,0])))
    real_mse.append(mean(array(mse)))
    real_kmse.append(mean(array(kmse)))

    
import matplotlib.pyplot as plt
plt.plot(array(real_kmse)/array(real_mse))
plt.show()
print(real_kmse)
print(real_mse)  
print(array(real_kmse)/array(real_mse))
        
        
        